In [1]:
import torch
import pickle
import pandas as pd
import numpy as np
from src.train import *
from src.processing import *
from src.models import *
from src.inference import *
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertModel
from torch.utils.data import Dataset, DataLoader

torch.manual_seed(42)
np.random.seed(42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

torch.cuda.empty_cache()

Using device: cuda


In [2]:
vocab = Vocabulary()
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')
ratings_df, movie_descriptions, movies_metadata = create_ratings_df(
    number_of_movies=7500,
    frac=0.01,
    links_path='CLIP4Rec/archive/links.csv',
    movies_metadata_path='CLIP4Rec/archive/movies_metadata.csv',
    ratings_path='CLIP4Rec/archive/ratings.csv'
    )
sequences = get_sequences(ratings_df)
vocab.build_vocab(sequences)

train_sentences, val_sentences = train_test_split(sequences, test_size=0.2, random_state=42)
train_data, film_descriptions_encoded = prepare_dataset(
    train_sentences, movie_descriptions, tokenizer, vocab, encode_descriptions=True
)
val_data = prepare_dataset(
    val_sentences, movie_descriptions, tokenizer, vocab
)

train_dataset = FilmRecommendationDataset(train_data, film_descriptions_encoded)
val_dataset = FilmRecommendationDataset(val_data, film_descriptions_encoded)

epochs = 2
batch_size = 24
lr = 0.0001

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


/teamspace/studios/this_studio/CLIP4Rec/src/processing.py:20: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies_metadata = pd.read_csv(movies_metadata_path)
100%|██████████| 2212/2212 [00:00<00:00, 452046.41it/s]


In [3]:
film_encoder = SASFilmEncoder(item_num=len(vocab.word_to_index), seq_len=seq_len, embed_dim=384, device=device)
text_encoder = TextEncoder(bert_model, output_dim=384)

In [4]:
train_clip(film_encoder, text_encoder, train_loader, val_loader, 
           epochs=epochs, lr=lr, device=device, iter_verbose=50)

  0%|          | 0/369 [00:00<?, ?it/s]

In [5]:
# list_movies = ["Only Lovers Left Alive",
#                "The Twilight Saga: Eclipse",
#                "Me Before You",
#                "(500) Days of Summer"]

In [3]:
# vocab = pd.read_pickle('CLIP4Rec/artifacts/vocab.pickle')
# movies_metadata = pd.read_pickle('CLIP4Rec/artifacts/movies_metadata.pickle')
# film_descriptions_encoded = pd.read_pickle('CLIP4Rec/artifacts/film_descriptions_encoded.pickle')
# bert_model = BertModel.from_pretrained('bert-base-uncased')
# bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# inference = Inference(
#     film_encoder_path = 'CLIP4Rec/artifacts/film_encoder_weights.pth',
#     text_encoder_path = 'CLIP4Rec/artifacts/text_encoder_weights.pth',
#     vocab=vocab,
#     dim=468,
#     movies_metadata=movies_metadata,
#     seq_len=seq_len,
#     device=device,
#     bert_model=bert_model,
#     bert_tokenizer=bert_tokenizer,
#     hidden_dim=256
# )
# inference.build_annoy_model(film_descriptions_encoded)

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
